## LOAD SOLUTION FILES

In [4]:
import os
import pickle
import pandas as pd
from attention import *
from helpers import *

In [255]:
from torch.autograd import Variable

def nopeak_mask(size):
    """Tạo mask được sử dụng trong decoder để lúc dự đoán trong quá trình huấn luyện
     mô hình không nhìn thấy được các từ ở tương lai
    """
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    # np_mask = np_mask.to(device)

    return np_mask

def create_masks(src, trg):
    """ Tạo mask cho encoder,
    để mô hình không bỏ qua thông tin của các kí tự PAD do chúng ta thêm vào
    """
    src_mask = (src != 0).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != 0).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size)
        if trg.is_cuda:
            np_mask.cuda()
        trg_mask = trg_mask & np_mask

    else:
        trg_mask = None
    return src_mask, trg_mask

in_mask, out_mask = create_masks(T_demand, Used_path)
in_mask.shape

torch.Size([1, 25, 1, 25])

### Extract OD demand and path set (X and Y)
X: OD demand, graph (link feature), path, link-path adj \
Y: path flow

In [1]:
%run helpers.py
%run attention.py

class Dataset():
    def __init__(self, size, input_dim0, input_dim1, output_dim0, output_dim1, start_from=0):
        super().__init__()
        self.path_encoded = path_encoder() # Get path encode dictionary
        self.entries = size
        self.X = torch.zeros([size, input_dim0, input_dim1], dtype=torch.float32)
        self.Y = torch.zeros([size, output_dim0, output_dim1], dtype=torch.float32)

        for i in tqdm(range(size)) :
            file_name = f"Output/5by5_Data{start_from+i}"
            x, y = generate_xy(file_name, self.path_encoded)
            self.X[i] = x
            self.Y[i] = y
    
    def __len__(self):
        return self.entries

    def __getitem__(self, idx):
        data_point = self.X[idx]
        data_label = self.Y[idx]
        return data_point, data_label
    

In [ ]:
# %run helpers.py

# Load first 1000 sample data
train_size = 10
train_dataset = Dataset(train_size, 625, 1165, 625, 3)
# this code create batch data, shape batch_size x seq_length x dim
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)

### Test Encoder

In [77]:
# print(torch.cuda.is_available())
size = 30
path_encoded = path_encoder()
X = torch.zeros([size, 625, 1165], dtype=torch.float32)
Y = torch.zeros([size, 625, 3], dtype=torch.float32)
for i in tqdm(range(size)) :
    file_name = f"Output/5by5_Data{i}"
    x, y = generate_xy(file_name, path_encoded)
    X[i] = x 
    Y[i] = y 

  0%|          | 0/30 [00:00<?, ?it/s]

In [70]:
torch.flatten(X, start_dim=1).shape

torch.Size([30, 728125])

In [47]:
a = torch.matmul(X, X.transpose(-2, -1))/math.sqrt(X.size(-1))
a.shape

torch.Size([30, 625, 625])

In [64]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

q = torch.randint(0, 500, (30, 8, 625, 64)).float()
k = torch.randint(0, 500, (30, 8, 625, 64)).float()
v = torch.randint(0, 500, (30, 8, 625, 64)).float()
mask_tensor = mask_tensor.unsqueeze(1)

values, attention = scaled_dot_product(q, k, v, mask_tensor)


In [84]:
%run Transformer1.py
%run attention.py

def expand_mask(mask):
    for i in range(mask.shape[0]):
        # Mở rộng mask trên chiều thứ tư
        mask[i] = mask[i].expand(mask.shape[2], mask.shape[2])
    return mask

# print("input shape: ", X.shape)
X = torch.flatten(X, start_dim=1)
print("input shape: ", X.shape)
mask_tensor = torch.rand(X.shape[0], 1, X.shape[-1]).float()
mask_tensor = expand_mask(mask_tensor.unsqueeze(1))
# mask_tensor = (X!=0).unsqueeze(1)
print("Mask shape: ", mask_tensor.shape)

# transformer = TransformerEncoder(num_layers=4,input_dim=1165,dim_feedforward=512,num_heads=8, dropout=0.1)
# x2 = transformer(X, mask_tensor)
# x2 = Encoder(input_dim=X.shape[-1], d_model=512, N=2, heads=8, dropout=0.1)(X, mask_tensor)
# print("Encoder output shape: ", x2.shape)

input shape:  torch.Size([30, 728125])


RuntimeError: expand(torch.FloatTensor{[1, 1, 728125]}, size=[1, 1]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

### Test Decoder

In [61]:
%run attention.py
decoder = Decoder(3, 512, 2, 8, 0.1)
x3 = decoder(Y, x2, mask_tensor)
x3.shape

Score shape:  torch.Size([30, 8, 625, 625])
V shape:  torch.Size([30, 8, 625, 64])
Score shape:  torch.Size([30, 8, 625, 625])
V shape:  torch.Size([30, 8, 625, 64])
Score shape:  torch.Size([30, 8, 625, 625])
V shape:  torch.Size([30, 8, 625, 64])
Score shape:  torch.Size([30, 8, 625, 625])
V shape:  torch.Size([30, 8, 625, 64])


torch.Size([30, 625, 512])